<a href="https://colab.research.google.com/github/Gabuni19/Codigos-Shorts/blob/main/Metodo_Simplex_Dual_Lineal_Entera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#METODO SIMPLEX

In [34]:
!pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 35.9 MB/s eta 0:00:00


In [35]:
from pulp import *

In [2]:

# Definir los datos del problema
c = [15.59,143.23,17.82]
A = [[0.048,0.0484,0.0051],
     [0.1364,1.3636,0.1364],
     [0.1364,1.3636,0.1364],
     [0.2045,2.0455,0.1705],
     [0.1461,1.4610,0.1461],
     [0.1000, 0.6666, 0.1000],
     [0.0667,0.2000,0.0500],
     [0.1500,0.9999,0.1500],
     [0.0500,0.1000,0.0500],
     [0.0528,0.3333,0.0333],
     [0.1000, 0.1666, 0.1000],
     [0.1000, 0.1000, 0.1000],
     [0.067,0.1333,0.0667],
     [1.0,0.0,0.0]]
b = [300.0,2400.0, 1200.0,4800.0,
     3000.0,3000.0,3000.0,
     3000.0,3000.0,3000.0,
     3000.0,3000.0,3000.0,
     21714.0]


In [155]:
import numpy as np
from tabulate import tabulate
def Transformar(c,A,b):
  E = []
  F = np.array(c)
  for i in range(np.shape(A)[0]):
    fila = np.concatenate((A[i], [b[i]]))
    E.append(fila)
  E = np.array(E)
  print(E)
  print(F)
  return E,F
def MetodoSimplex(c,A,b):
  print(A)
  E,F = Transformar(c,A,b)

  #Calculamos variables auxiliares
  n_variables = np.shape(F)[0]
  n_ecuaciones =  np.shape(E)[0]
  print("Numero de variables: {}".format(n_variables))
  print("Numero de ecuaciones: {}\n".format(n_ecuaciones))

  # Creamos el arreglo que contiene el indice de las variables de la columna r
  indice_var = np.array([-1 for i in range(n_ecuaciones)])
  print(indice_var)

  # Paso1 :Creacion de la matriz
  n_filas = n_ecuaciones+1
  n_columnas =  (n_variables) + (n_ecuaciones)+2
  M = np.zeros((n_filas,n_columnas))
  print(f"M : ({n_filas},{n_columnas})")
  print(M)

  # Paso 2 : Rellenamos la matriz
  # 1) Rellanamos la ultima fila
  M[-1,0]=1
  M[-1,1:n_variables+1] = F*(-1)


  # 2) Rellenamos las demas filas
  for i in range(n_ecuaciones):
    M[i, 1:n_variables + 1] = E[i, :n_variables]  # Rellenando las variables no básicas
    M[i, n_variables + 1 + i] = 1  # Rellenamos la diagonal 1 si hay más variables que ecuaciones
    M[i, -1] = E[i, -1]  # Rellenando el resultado

  # Establecer opciones de impresión para NumPy
  np.set_printoptions(precision=2, suppress=True)
  header = [""] + ["x" + str(i + 1) for i in range(n_variables)] + ["s" +  str(i+1) for i in range(n_ecuaciones)] + ["RHS"]
  print(tabulate(M, headers=header, tablefmt="plain"))


  while True :
    # Paso 3 : Seleccionamos de la fila z la columna q
    fila_z  = M[-1,1:n_variables+1]

    mayor=-1
    columna_q =0
    for i in range(n_variables):
      if(fila_z[i]<0 and abs(fila_z[i])>mayor):
        mayor=i
        columna_q=i+1

    print(tabulate([[fila_z,fila_z[columna_q-1],columna_q]],headers = ["Fila z (NO BASICA)","Mayor","Columna Escogida"], tablefmt="grid", floatfmt=".4f"))

    # Paso 4: Calcula las razones
    fila_resultado = np.zeros(n_ecuaciones)
    aiq = M[:n_ecuaciones, columna_q]

    # Comprobación de división entre cero
    for i in range(n_ecuaciones):
        if aiq[i] != 0:
            fila_resultado[i] = M[i, -1] / aiq[i]
        else:
            fila_resultado[i] = float('inf')  # Asigna infinito en caso de división entre cero

    no_acotado = True
    for a in aiq:
      if a > 0:
        no_acotado = False

    if(no_acotado):
      print(" [-] El problema no esta acotado.")
      break

    # Paso 5 :Se elige la menor razon positiva
    menor =99999
    fila_q=0
    for i in range(len(fila_resultado)):
      if(fila_resultado[i]>0 and fila_resultado[i]<menor):
        menor = fila_resultado[i]
        fila_q=i

    #Asignamos un valor al arreglo de indices
    indice_var[fila_q] = columna_q

    print(tabulate([[fila_resultado,fila_resultado[fila_q],fila_q]],headers = ["Fila Razones (bi/aiq)","Menor","Fila Escogida"], tablefmt="grid", floatfmt=".4f"))

    # Paso 6 : Seleccionamos el pivote
    pivote = M[fila_q,columna_q]
    print("\n -> Pivote: {}".format(pivote))

    # Paso 7 : Aplicamos operaciones elementales
    # Hacemos 1 el pivote
    for i in range(n_columnas):
      M[fila_q,i] /= pivote

    pivote = M[fila_q,columna_q]

    print(" -> (Fila {}) / pivote".format(fila_q))
    print(" -> Nuevo pivote : {}".format(pivote))
    print(tabulate(M, headers=header, tablefmt="plain"))
    print(" [+] Hacemos 0 la (Columna {})".format(columna_q))
    # y hacemos 0 la columna q
    for i in range(n_filas):
      if i != fila_q:
        factor = -M[i,columna_q]
        for j in range(n_columnas):
          M[i,j] = M[i,j] + factor*M[fila_q,j]

    print(tabulate(M, headers=header, tablefmt="plain"))

    # Paso 8 : Si todos los coeficientes de la fila z en cuyas columnas estan
    # x1, x2, , xn son positivas o ceros, se para. Sino se regresa al paso 3 y se
    # repite el proceso.
    if all(x >= 0 for x in M[-1, 1:n_variables + 1]):
      break

  print(tabulate(M, headers=header, tablefmt="plain"))
  # Formatear la matriz para que los números tengan dos decimales
  M_formatted = [[round(cell, 4) if isinstance(cell, float) else cell for cell in row] for row in M]
  columna_r =  M[:,-1]
  dic = {"FO":columna_r[-1],"variables":[]}
  print(" Arreglo indices:",indice_var)
  k=0
  l = 0
  for i in range(len(indice_var)):
    if indice_var[i]!=-1:
      print(f" -> x{indice_var[i]} = {columna_r[i]}")
      dic["variables"].append((indice_var[i],columna_r[i]))

    else:
      if(l==0):
        print(f" -> x (rest) = 0")
        l+=1

  print(" [+] Maximo Valor =  {}".format(columna_r[-1]))
  return dic



In [156]:
def Transformar(c,A,b):
  E = []
  F = np.array(c)
  for i in range(np.shape(A)[0]):
    fila = np.concatenate((A[i], [b[i]]))
    E.append(fila)
  E = np.array(E)
  print(E)
  print(F)
  return E,F
def MetodoSimplex1(c,A,b):
  #print(A)
  E,F = Transformar(c,A,b)

  #Calculamos variables auxiliares
  n_variables = np.shape(F)[0]
  n_ecuaciones =  np.shape(E)[0]
  #print("Numero de variables: {}".format(n_variables))
  #print("Numero de ecuaciones: {}\n".format(n_ecuaciones))

  # Creamos el arreglo que contiene el indice de las variables de la columna r
  indice_var = np.array([-1 for i in range(n_ecuaciones)])
  #print(indice_var)

  # Paso1 :Creacion de la matriz
  n_filas = n_ecuaciones+1
  n_columnas =  (n_variables) + (n_ecuaciones)+2
  M = np.zeros((n_filas,n_columnas))
  #print(f"M : ({n_filas},{n_columnas})")
  #print(M)

  # Paso 2 : Rellenamos la matriz
  # 1) Rellanamos la ultima fila
  M[-1,0]=1
  M[-1,1:n_variables+1] = F*(-1)


  # 2) Rellenamos las demas filas
  for i in range(n_ecuaciones):
    M[i, 1:n_variables + 1] = E[i, :n_variables]  # Rellenando las variables no básicas
    M[i, n_variables + 1 + i] = 1  # Rellenamos la diagonal 1 si hay más variables que ecuaciones
    M[i, -1] = E[i, -1]  # Rellenando el resultado

  # Establecer opciones de impresión para NumPy
  np.set_printoptions(precision=2, suppress=True)
  header = [""] + ["x" + str(i + 1) for i in range(n_variables)] + ["s" +  str(i+1) for i in range(n_ecuaciones)] + ["RHS"]
  #print(tabulate(M, headers=header, tablefmt="plain"))


  while True :
    # Paso 3 : Seleccionamos de la fila z la columna q
    fila_z  = M[-1,1:n_variables+1]

    mayor=-1
    columna_q =0
    for i in range(n_variables):
      if(fila_z[i]<0 and abs(fila_z[i])>mayor):
        mayor=i
        columna_q=i+1

    #print(tabulate([[fila_z,fila_z[columna_q-1],columna_q]],headers = ["Fila z (NO BASICA)","Mayor","Columna Escogida"], tablefmt="grid", floatfmt=".4f"))

    # Paso 4: Calcula las razones
    fila_resultado = np.zeros(n_ecuaciones)
    aiq = M[:n_ecuaciones, columna_q]

    # Comprobación de división entre cero
    for i in range(n_ecuaciones):
        if aiq[i] != 0:
            fila_resultado[i] = M[i, -1] / aiq[i]
        else:
            fila_resultado[i] = float('inf')  # Asigna infinito en caso de división entre cero

    no_acotado = True
    for a in aiq:
      if a > 0:
        no_acotado = False

    if(no_acotado):
      print(" [-] El problema no esta acotado.")
      break

    # Paso 5 :Se elige la menor razon positiva
    menor =99999
    fila_q=0
    for i in range(len(fila_resultado)):
      if(fila_resultado[i]>0 and fila_resultado[i]<menor):
        menor = fila_resultado[i]
        fila_q=i

    #Asignamos un valor al arreglo de indices
    indice_var[fila_q] = columna_q

    #print(tabulate([[fila_resultado,fila_resultado[fila_q],fila_q]],headers = ["Fila Razones (bi/aiq)","Menor","Fila Escogida"], tablefmt="grid", floatfmt=".4f"))

    # Paso 6 : Seleccionamos el pivote
    pivote = M[fila_q,columna_q]
    #print("\n -> Pivote: {}".format(pivote))

    # Paso 7 : Aplicamos operaciones elementales
    # Hacemos 1 el pivote
    for i in range(n_columnas):
      M[fila_q,i] /= pivote

    pivote = M[fila_q,columna_q]

    #print(" -> (Fila {}) / pivote".format(fila_q))
    #print(" -> Nuevo pivote : {}".format(pivote))
    #print(tabulate(M, headers=header, tablefmt="plain"))
    #print(" [+] Hacemos 0 la (Columna {})".format(columna_q))
    # y hacemos 0 la columna q
    for i in range(n_filas):
      if i != fila_q:
        factor = -M[i,columna_q]
        for j in range(n_columnas):
          M[i,j] = M[i,j] + factor*M[fila_q,j]

    #print(tabulate(M, headers=header, tablefmt="plain"))

    # Paso 8 : Si todos los coeficientes de la fila z en cuyas columnas estan
    # x1, x2, , xn son positivas o ceros, se para. Sino se regresa al paso 3 y se
    # repite el proceso.
    if all(x >= 0 for x in M[-1, 1:n_variables + 1]):
      break

  #print(tabulate(M, headers=header, tablefmt="plain"))
  # Formatear la matriz para que los números tengan dos decimales
  M_formatted = [[round(cell, 4) if isinstance(cell, float) else cell for cell in row] for row in M]
  columna_r =  M[:,-1]
  dic = {"FO":columna_r[-1],"variables":[]}
  #print(" Arreglo indices:",indice_var)
  k=0
  l = 0
  for i in range(len(indice_var)):
    if indice_var[i]!=-1:
      #print(f" -> x{indice_var[i]} = {columna_r[i]}")
      dic["variables"].append((indice_var[i],columna_r[i]))

    else:
      if(l==0):
        #print(f" -> x (rest) = 0")
        l+=1

  #print(" [+] Maximo Valor =  {}".format(columna_r[-1]))
  return dic

#METODO DUAL

In [157]:
import numpy as np

def DualSimplex(c, A, b, tipo):
    if tipo == "minimizar":
        c_dual = np.array(b)
        b_dual = np.array(c)
        A_dual = np.transpose(np.array(A))
        MetodoSimplex(c_dual, A_dual, b_dual)
    elif tipo == "maximizar":
        MetodoSimplex(c, A, b)
    else:
        print("Escoge entre maximizar y minimizar")

#*****MINIMIZAR***************
# Definir los datos del problema
A = [[0.0048, 0.0484, 0.0051],
         [0.1364, 1.3636, 0.1364],
         [0.1364, 1.3636, 0.1364],
         [0.2045, 2.0455, 0.1705],
         [0.1461, 1.4610, 0.1461],
         [0.1000, 0.6666, 0.1000],
         [0.0667, 0.2000, 0.0500],
         [0.1500, 0.9999, 0.1500],
         [0.0500, 0.1000, 0.0500],
         [0.0528, 0.3333, 0.0333],
         [0.1000, 0.1666, 0.1000],
         [0.1000, 0.1000, 0.1000],
         [0.0667, 0.1333, 0.0667],
         [1.0000, 0.0000, 0.0000]]
b = [300, 2400, 1200, 4800, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 3000, 21714]
c = [15.59, 143.23, 17.82]

#b = [300,2400,1200,4800,3000,3000,3000,3000,3000,3000,3000,3000,3000,21714]

# Resolver el problema utilizando el método simplex

DualSimplex(c,A,b,"minimizar")
DualSimplex(c,A,b,"maximizar")


[[0.   0.14 0.14 0.2  0.15 0.1  0.07 0.15 0.05 0.05 0.1  0.1  0.07 1.  ]
 [0.05 1.36 1.36 2.05 1.46 0.67 0.2  1.   0.1  0.33 0.17 0.1  0.13 0.  ]
 [0.01 0.14 0.14 0.17 0.15 0.1  0.05 0.15 0.05 0.03 0.1  0.1  0.07 0.  ]]
[[  0.     0.14   0.14   0.2    0.15   0.1    0.07   0.15   0.05   0.05
    0.1    0.1    0.07   1.    15.59]
 [  0.05   1.36   1.36   2.05   1.46   0.67   0.2    1.     0.1    0.33
    0.17   0.1    0.13   0.   143.23]
 [  0.01   0.14   0.14   0.17   0.15   0.1    0.05   0.15   0.05   0.03
    0.1    0.1    0.07   0.    17.82]]
[  300  2400  1200  4800  3000  3000  3000  3000  3000  3000  3000  3000
  3000 21714]
Numero de variables: 14
Numero de ecuaciones: 3

[-1 -1 -1]
M : (4,19)
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
           x1          x2          x3          x4 

/usr/local/lib/python3.10/dist-packages/tabulate/__init__.py:107: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  (len(row) >= 1 and row[0] == SEPARATING_LINE)


In [158]:
import pulp

def MetodoSimplexPuLP(c, A, b,obj = "max"):
    """
    Resuelve un problema de programación lineal mixta (MILP) utilizando PuLP y el solucionador CBC.

    Args:
        c (list): Coeficientes de la función objetivo.
        A (list of lists): Coeficientes de las restricciones (matriz A).
        b (list): Lados derecho de las restricciones (vector b).

    Returns:
        tuple: Una tupla que contiene el estado de la solución, el valor óptimo y las soluciones de las variables.
    """
    # Crear un problema de PuLP
    prob = pulp.LpProblem("MaximizationProblem", pulp.LpMaximize)  # Modificamos el nombre del problema

    # Crear variables
    n = len(c)
    x = [pulp.LpVariable(f"x{i}", lowBound=0) for i in range(n)]  # Utilizamos nombres de variables más simples

    # Definir la función objetivo
    prob += pulp.lpSum(c[i] * x[i] for i in range(n)), "ObjectiveFunction"

    # Definir las restricciones
    for i in range(len(A)):
        prob += pulp.lpSum(A[i][j] * x[j] for j in range(n)) <= b[i], f"Constraint{i}"  # Utilizamos nombres de restricciones más simples

    # Resolver el problema con el solucionador CBC
    prob.solve(pulp.PULP_CBC_CMD())

    # Obtener el estado, el valor óptimo y las soluciones de las variables
    estado = pulp.LpStatus[prob.status]
    valor_optimo = pulp.value(prob.objective)
    soluciones = [x[i].varValue for i in range(n)]

    return valor_optimo, soluciones


In [159]:
import pulp

def AlgoritmoSimplex(c, A, b, objetivo="max"):
    """
    Resuelve un problema de programación lineal mixta (MILP) utilizando PuLP y el solucionador CBC.

    Args:
        c (list): Coeficientes de la función objetivo.
        A (list of lists): Coeficientes de las restricciones (matriz A).
        b (list): Lados derecho de las restricciones (vector b).
        objetivo (str): Cadena que indica si se debe "maximizar" o "minimizar" (predeterminado: "max").

    Returns:
        tuple: Una tupla que contiene el estado de la solución, el valor óptimo y las soluciones de las variables.
    """
    # Verificar el objetivo y crear un problema de PuLP correspondiente
    if objetivo.lower() == "max":
        prob = pulp.LpProblem("MaximizationProblem", pulp.LpMaximize)
    elif objetivo.lower() == "min":
        prob = pulp.LpProblem("MinimizationProblem", pulp.LpMinimize)
    else:
        raise ValueError("El parámetro 'objetivo' debe ser 'max' o 'min'")

    # Crear variables
    n = len(c)
    x = [pulp.LpVariable(f"x{i}", lowBound=0) for i in range(n)]

    # Definir la función objetivo
    if objetivo.lower() == "max":
        prob += pulp.lpSum(c[i] * x[i] for i in range(n)), "ObjectiveFunction"
    elif objetivo.lower() == "min":
        prob += pulp.lpSum(-c[i] * x[i] for i in range(n)), "ObjectiveFunction"

    # Definir las restricciones
    for i in range(len(A)):
        prob += pulp.lpSum(A[i][j] * x[j] for j in range(n)) <= b[i], f"Constraint{i}"

    # Resolver el problema con el solucionador CBC
    prob.solve(pulp.PULP_CBC_CMD())

    # Obtener el estado, el valor óptimo y las soluciones de las variables
    estado = pulp.LpStatus[prob.status]
    if objetivo.lower() == "max":
        valor_optimo = pulp.value(prob.objective)
    elif objetivo.lower() == "min":
        valor_optimo = -pulp.value(prob.objective)
    soluciones = [x[i].varValue for i in range(n)]

    return valor_optimo, soluciones


In [160]:
# Ejemplo de uso
c2 = [1, 5]
A2 = [[11, 6], [5, 50]]
b2 = [66, 225]

valor_optimo, soluciones = AlgoritmoSimplex(c2, A2, b2)
print(f"Valor óptimo: {valor_optimo}")
print(f"Soluciones: {soluciones}")

Valor óptimo: 24.375
Soluciones: [3.75, 4.125]


#Programacion Lineal Entera

In [161]:
import numpy as np
from tabulate import tabulate
from scipy.optimize import linprog
import math

def PLE(c, A, b, obj="max"):

    # Paso 1: Resolver el problema como un modelo continuo
    print("==================================================")
    print(f"Resolviendo como modelo continuo:")
    print(tabulate([[A, b, c]], headers=["A", "b", "c"], tablefmt="grid"))

    f0,v0 = AlgoritmoSimplex(c, A, b,obj)
    print(f"Solución  = {f0} -  {v0}")

    decimal = None
    tupla_decimal = None

    # Buscar el primer valor decimal
    for i, valor in enumerate(v0):
      if not valor.is_integer():
          tupla_decimal = (i+1, valor)
          decimal = valor
          break

    # Luego del for, se tiene la tupla con valor decimal

    if decimal is not None:
        n_var = tupla_decimal[0]
        print(tabulate([[v0]], headers=["variables"], tablefmt="grid"))
        print(f"\nTupla decimal: {tupla_decimal}")
        A1 = A.copy()
        b1 = b.copy()
        A2 = A.copy()
        b2 = b.copy()

        n_variables = len(A[0])
        #print(f"Número de variables: {n_variables}")

        e1 = [0 for i in range(n_variables)]
        e1[n_var - 1] = 1
        b1.append(math.floor(decimal))
        A1.append(e1)

        print("==================================================")
        print(f"Subproblema 1: xj <= {math.floor(decimal)}")
        print(tabulate([[A1, b1, c]], headers=["A", "b", "c"], tablefmt="grid"))

        f1, v1 = AlgoritmoSimplex(c, A1, b1, obj)
        print(f"Valor óptimo 1: {f1}")
        print(f"Soluciones 1: {v1}")



        print("==================================================")
        print(f"Subproblema 2: xj >= {math.floor(decimal) + 1}")
        e2 = [0 for i in range(n_variables)]
        e2[n_var - 1] = -1
        b2.append(-(math.floor(decimal) + 1))
        A2.append(e2)

        print(tabulate([[A2, b2, c]], headers=["A", "b", "c"], tablefmt="grid"))

        f2, v2 = AlgoritmoSimplex(c, A2, b2, obj)
        print(f"Valor óptimo 2: {f2}")
        print(f"Soluciones 2: {v2}")

        # Recoger los valores óptimos
        valor_escogido = max([f1, f2])
        print(f"\n-> Valor escogido: {valor_escogido}")

        # Caso base para detener la recursión
        if f1.is_integer() and f2.is_integer():
            print("Ambas soluciones son enteras. Terminando.")
            return

        # Llamar a la función PLE recursivamente con las nuevas instancias
        if valor_escogido == f1:
            PLE(c, A1, b1, obj)
            return
        elif valor_escogido == f2:
            PLE(c, A2, b2, obj)
            return
    else:
      print("Se encontro solucion Entera")





In [154]:
A = [[11,6],
     [5,50]
     ]
b = [66,225]
c = [1,5]
d = PLE(c,A,b)


Resolviendo como modelo continuo:
+--------------------+-----------+--------+
| A                  | b         | c      |
+====================+===========+========+
| [[11, 6], [5, 50]] | [66, 225] | [1, 5] |
+--------------------+-----------+--------+
Solución  = 24.375 -  [3.75, 4.125]
+---------------+
| variables     |
+===============+
| [3.75, 4.125] |
+---------------+

Tupla decimal: (1, 3.75)
Subproblema 1: xj <= 3
+----------------------------+--------------+--------+
| A                          | b            | c      |
+============================+==============+========+
| [[11, 6], [5, 50], [1, 0]] | [66, 225, 3] | [1, 5] |
+----------------------------+--------------+--------+
Valor óptimo 1: 24.0
Soluciones 1: [3.0, 4.2]
Subproblema 2: xj >= 4
+-----------------------------+---------------+--------+
| A                           | b             | c      |
+=============================+===============+========+
| [[11, 6], [5, 50], [-1, 0]] | [66, 225, -4] | [1, 5] |